In [1]:
import os
from dotenv import load_dotenv
import asyncio

from autogen_agentchat.agents import AssistantAgent,UserProxyAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination

from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


# Load environment variables from a .env file
load_dotenv("/home/azureuser/azure-ai-agent-workshop/.env")

True

In [2]:
model_client=AzureOpenAIChatCompletionClient(
            model=os.getenv("AZURE_OPENAI_MODEL"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_KEY"),
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            api_version=os.getenv("AZUER_OPENAI_API_VERSION")
        )

In [3]:
# Define a function to simulate user feedback
def custom_input(task):
    return "It is raning in shanghai. I am feeling sad."

In [4]:
# define Agent
weather_agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    description="an agent with human feedback"
)

# define Agent
user_proxy = UserProxyAgent(
    name="user_proxy",
    input_func=custom_input
)

In [5]:
# Define a termination condition. End the conversation if a specific text is mentioned.
text_termination = TextMentionTermination("<stop autogenworkshop>")
# Define a termination condition: stop the task after 5 messages
max_message_termination = MaxMessageTermination(4)
# Use the `|` operator to combine termination conditions, stopping the task if any condition is met
termination = text_termination | max_message_termination

In [6]:
# Define Team with type set to RoundRobinGroupChat
agent_team = RoundRobinGroupChat(participants=[weather_agent,user_proxy], termination_condition=termination, max_turns=None)


In [7]:
# Run the team and use the official Console tool to output in the proper format
stream = agent_team.run_stream(task="what's weather of Shanghai?")
await Console(stream)

---------- user ----------
what's weather of Shanghai?


---------- weather_agent ----------
I don't have real-time internet access to provide current weather information. However, you can easily check the current weather in Shanghai using a weather website or app. Would you like any other assistance?
---------- user_proxy ----------
It is raning in shanghai. I am feeling sad.
---------- weather_agent ----------
I'm sorry to hear that you're feeling sad, especially with the rainy weather. Sometimes gloomy weather can affect our mood. If you’d like, I can suggest some activities to lift your spirits or provide some comforting words. Let me know how I can help!


TaskResult(messages=[TextMessage(source='user', models_usage=None, content="what's weather of Shanghai?", type='TextMessage'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=42, completion_tokens=38), content="I don't have real-time internet access to provide current weather information. However, you can easily check the current weather in Shanghai using a weather website or app. Would you like any other assistance?", type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, request_id='1916aa0d-e423-4135-a456-b419dc8756f4', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, content='It is raning in shanghai. I am feeling sad.', type='TextMessage'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=106, completion_tokens=51), content="I'm sorry to hear that you're feeling sad, especially with the rainy weather. Sometimes gloomy weather can affect our mood. If 